# Lesson 4: Learning Representations + Style Transfer

Look into what a network learns

Examine Representations (RSMs)

Extract these for content loss

Style loss: Gramm matrix version

Style loss: OT version

## Aside: NBDEV and exporting functions for later use

## Aside: Making Gradio Demos (and use as exercise)

## Aside: Imstack

In [ ]:
# TODO: Include AEs/VAEs? 
# TODO: Contrastive loss anywhere?